# Linear Solver Performance: 2D Stokes manufactured Solution after Botti and Di Pietro
### Part 1, Benchmark Setup and Execution

This benchmark was proposed in a work:

 "p‑Multilevel Preconditioners for HHO Discretizations of the Stokes Equations with Static Condensation" 
 by L. Botti and D. Di Pietro (https://doi.org/10.1007/s42967-021-00142-5)
 
It is used to assess DG as well as HDG methods, the latter beeing the main focus of the paper, the former only for reference.
The exact solution to the stationary Stokes equation is
$$
u_1 =  -\exp(x)*(y*\cos(y) + \sin(y)), \\
u_2 =  \exp(x)*y*\sin(y), \\
  p =  2*\exp(x)*\sin(y) .
$$
The domain is set as $ \Omega = (-1,1) $.
This exact solution is also used as a boundary condition on three sides; On one side, a Neumann boundary condition is enforced. It is not mentioned which side this is. 


### Note

This example can be found in the source code repository as as `LinslvPerf_BottiPietroStokes2D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 4/8/2022 8:53:23 AM


In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
wmg.Init("LinslvPerfSer");
wmg.AllJobs

Project name is set to 'LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerfSer'.


In [4]:
// for examination on the local workstation, 
//OpenDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes");

In [5]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [6]:
//wmg.DefaultDatabase.Grids.ForEach(s => s.Delete(true));

## Utility definitions

In [7]:
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        return (p*p + 3*p + 2)/2; // 2D 
        //return (p*p*p + 6*p*p + 11*p + 6)/6; // 3D
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

In [8]:
int[] Resolutions_2D = new int[] { 8, 16, 24, 32, 48, 64, 128, 192, 256, 384, 512 };

IGridInfo[] grids = new IGridInfo[Resolutions_2D.Length];
for(int cnt = 0; cnt < Resolutions_2D.Length; cnt++) {
    int Res = Resolutions_2D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-Stokes2D_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid2D.Cartesian2DGrid(xNodes, yNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            if(Math.Abs(x - (-1)) < 1e-8)
                return "pressure_outlet";
            return "wall"; 
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 2)
            throw new Exception("D mismatch");
    }
}

Opening existing database 'C:\BoSSStests\LinslvPerfSer'.
Found Grid: { Guid = 3e3cd419-2ff0-4b99-be41-b04d46899299; Name = LinslvPerfSer-Stokes2D_J64; Cell Count = 64; Dim = 2 }
Found Grid: { Guid = 729e71b0-8bfd-4dd3-ae85-83bc9e431d0e; Name = LinslvPerfSer-Stokes2D_J256; Cell Count = 256; Dim = 2 }
Found Grid: { Guid = aa681b03-401e-4d5c-9860-6a0735782332; Name = LinslvPerfSer-Stokes2D_J576; Cell Count = 576; Dim = 2 }
Found Grid: { Guid = 136f770b-cfe1-45c9-ae50-af6c13371d27; Name = LinslvPerfSer-Stokes2D_J1024; Cell Count = 1024; Dim = 2 }
Found Grid: { Guid = 37795355-b07a-4ee7-8f1a-76af185c7a6f; Name = LinslvPerfSer-Stokes2D_J2304; Cell Count = 2304; Dim = 2 }
Found Grid: { Guid = 9730322f-d362-4351-9bda-abf491f2ef30; Name = LinslvPerfSer-Stokes2D_J4096; Cell Count = 4096; Dim = 2 }
Found Grid: { Guid = dcbd25cc-388e-432c-940f-a7f052aee973; Name = LinslvPerfSer-Stokes2D_J16384; Cell Count = 16384; Dim = 2 }
Found Grid: { Guid = 087a8e53-0f11-4816-985d-d5e3b376c6f6; Name = LinslvPe

In [9]:
grids

#0: { Guid = 3e3cd419-2ff0-4b99-be41-b04d46899299; Name = LinslvPerfSer-Stokes2D_J64; Cell Count = 64; Dim = 2 }
#1: { Guid = 729e71b0-8bfd-4dd3-ae85-83bc9e431d0e; Name = LinslvPerfSer-Stokes2D_J256; Cell Count = 256; Dim = 2 }
#2: { Guid = aa681b03-401e-4d5c-9860-6a0735782332; Name = LinslvPerfSer-Stokes2D_J576; Cell Count = 576; Dim = 2 }
#3: { Guid = 136f770b-cfe1-45c9-ae50-af6c13371d27; Name = LinslvPerfSer-Stokes2D_J1024; Cell Count = 1024; Dim = 2 }
#4: { Guid = 37795355-b07a-4ee7-8f1a-76af185c7a6f; Name = LinslvPerfSer-Stokes2D_J2304; Cell Count = 2304; Dim = 2 }
#5: { Guid = 9730322f-d362-4351-9bda-abf491f2ef30; Name = LinslvPerfSer-Stokes2D_J4096; Cell Count = 4096; Dim = 2 }
#6: { Guid = dcbd25cc-388e-432c-940f-a7f052aee973; Name = LinslvPerfSer-Stokes2D_J16384; Cell Count = 16384; Dim = 2 }
#7: { Guid = 087a8e53-0f11-4816-985d-d5e3b376c6f6; Name = LinslvPerfSer-Stokes2D_J36864; Cell Count = 36864; Dim = 2 }
#8: { Guid = e4d05f49-5208-46a8-8a0c-cbfb8cd148a8; Name = LinslvPerf

In [10]:
//PlotGrid("g2304",grids[4]);

In [11]:
wmg.DefaultDatabase

{ Session Count = 267; Grid Count = 30; Path = \\fdygitrunner\ValidationTests\LinslvPerfSer }

## Setup Control Object for a Solver Run

In [12]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [13]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Boundary Conditions and Exact Solution

In [14]:
var VelocityX = new Formula("(X) => -Math.Exp(X[0])*(X[1]*Math.Cos(X[1]) + Math.Sin(X[1]))");

In [15]:
var VelocityY = new Formula("(X) => Math.Exp(X[0])*X[1]*Math.Sin(X[1])");

In [16]:
var Pressure = new Formula("(X) => 2*Math.Exp(X[0])*Math.Sin(X[1])");

In [17]:
NUnit.Framework.Assert.Less((VelocityX.Evaluate(new double[] { 1, 0.5 }, 0) - (-2.495972095)).Abs(), 1.0e-9, 
     "x-Velocity expression differs from reference value");

In [18]:
NUnit.Framework.Assert.Less((VelocityY.Evaluate(new double[] { 1, 0.5 }, 0) - 0.6516068645).Abs(), 1.0e-9, 
     "y-Velocity expression differs from reference value");

### Setup of Parameter Study

Polynomial degrees to test:

In [19]:
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [20]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [21]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [22]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [23]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*2 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("BottiPietroStokes2D-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    C.PhysicalParameters.rho_A             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.rho_B             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.mu_A              = 1; // dimensionless
    C.PhysicalParameters.mu_B              = 1; // dimensionless
    C.PhysicalParameters.Sigma             = 0; // not relevant, since single phase
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Boundary Conditions
    // ===================
    C.AddBoundaryValue("wall", "VelocityX", VelocityX);
    C.AddBoundaryValue("wall", "VelocityY", VelocityY);
    
    C.AddInitialValue("VelocityX", VelocityX);
    C.AddInitialValue("VelocityY", VelocityY);
   
    // Solver Stuff
    // ============
    
    //C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.LevelSet_ConvergenceCriterion     = 1e-6;
    C.NoOfMultigridLevels = 100;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

setting up: BottiPietroStokes2D-J64_p2_direct_pardiso
setting up: BottiPietroStokes2D-J256_p2_direct_pardiso
setting up: BottiPietroStokes2D-J576_p2_direct_pardiso
setting up: BottiPietroStokes2D-J1024_p2_direct_pardiso
setting up: BottiPietroStokes2D-J2304_p2_direct_pardiso
setting up: BottiPietroStokes2D-J4096_p2_direct_pardiso
setting up: BottiPietroStokes2D-J16384_p2_direct_pardiso
setting up: BottiPietroStokes2D-J36864_p2_direct_pardiso
setting up: BottiPietroStokes2D-J65536_p2_direct_pardiso
setting up: BottiPietroStokes2D-J64_p3_direct_pardiso
setting up: BottiPietroStokes2D-J256_p3_direct_pardiso
setting up: BottiPietroStokes2D-J576_p3_direct_pardiso
setting up: BottiPietroStokes2D-J1024_p3_direct_pardiso
setting up: BottiPietroStokes2D-J2304_p3_direct_pardiso
setting up: BottiPietroStokes2D-J4096_p3_direct_pardiso
setting up: BottiPietroStokes2D-J16384_p3_direct_pardiso
setting up: BottiPietroStokes2D-J36864_p3_direct_pardiso
setting up: BottiPietroStokes2D-J64_p5_direct_pardi

Total number of simulations:

In [24]:
controls.Count

105

In [25]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [29]:
//string path = @"C:\Users\jenkinsci\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Stokes2D";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, "control-" + ctrl.SessionName + ".obj"));
//}

## Launch Jobs

In [26]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch();
}

 Submitting: BottiPietroStokes2D-J64_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J64_p2_classic_pardiso	03/22/2022 09:03:17	5f8717de..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J256_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J256_p2_classic_pardiso	03/22/2022 09:03:50	a0b775a7..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J576_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J576_p2_classic_pardiso	03/22/2022 09:04:24	44cc0894..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J1024_p2_direct_pardiso
Loading session 879748d1-75b7-480d-b37f-acb5075d55ef failed with message 'Could not 

No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J2304_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J2304_p2_exp_gmres_levelpmg	03/22/2022 09:21:27	b07d3bf8..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J4096_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J4096_p2_exp_gmres_levelpmg	03/22/2022 09:22:12	d959a9ac..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg	03/22/2022 09:22:59	ee54b27a..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J36864_p2_exp_gmres_

 Submitting: BottiPietroStokes2D-J36864_p2_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J36864_p2_exp_Kcycle_schwarz	03/22/2022 09:48:22	668ba7b9..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J65536_p2_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J65536_p2_exp_Kcycle_schwarz	03/22/2022 09:49:24	2225a68d..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J147456_p2_exp_Kcycle_schwarz


IOException during Job.Deployment status evaluation: job.exe view 418823  /scheduler:DC2 exited with code 1

Microsoft.Hpc.Scheduler.Properties.SchedulerException: The specified Job ID is not valid. Check your Job ID and try again.
   at Microsoft.Hpc.Scheduler.Store.StoreServer.CallServerFuncWithErrorHandling(Func`1 action, Boolean asyncReconnect)
   at Microsoft.Hpc.Scheduler.Store.SchedulerStoreSvc.OpenJob(Int32 jobid)
   at CliTools.ViewJob.Execute(List`1 args)
   at CliTools.CommandVerbList.Execute(List`1 args)
   at CliTools.Program.RunList(CommandVerbList list, String[] args)

IOException during Job.Deployment status evaluation: job.exe view 418823  /scheduler:DC2 exited with code 1

Microsoft.Hpc.Scheduler.Properties.SchedulerException: The specified Job ID is not valid. Check your Job ID and try again.
   at Microsoft.Hpc.Scheduler.Store.StoreServer.CallServerFuncWithErrorHandling(Func`1 action, Boolean asyncReconnect)
   at Microsoft.Hpc.Scheduler.Store.SchedulerStoreSvc.Open

Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J147456_p2_exp_Kcycle_schwarz	03/22/2022 09:50:29	3ceef9bd..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J64_p3_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J64_p3_exp_Kcycle_schwarz	03/22/2022 09:51:33	093722e4..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J256_p3_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J256_p3_exp_Kcycle_schwarz	03/22/2022 09:52:35	3408df26..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J576_p3_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J576_p3_exp_Kcycle_schwarz	03/22/2022 09:53:38	302f

 Submitting: BottiPietroStokes2D-J576_p3_pMultigrid
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J576_p3_pMultigrid	03/22/2022 10:25:35	f3deb8ce..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J1024_p3_pMultigrid
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J1024_p3_pMultigrid	03/22/2022 10:26:53	21a4be20..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J2304_p3_pMultigrid
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J2304_p3_pMultigrid	03/22/2022 10:28:11	4f804b8d..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J4096_p3_pMultigrid
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J4096_p3_pMultigrid	03/22/2022 10:29:32	eac

In [33]:
wmg.AllJobs

### Wait for Completion and Check Job Status

In [27]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

All jobs finished.


In [28]:
wmg.AllJobs

#0: BottiPietroStokes2D-J64_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J64_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: BottiPietroStokes2D-J256_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J256_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: BottiPietroStokes2D-J576_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J576_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: BottiPietroStokes2D-J1024_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J1024_p2_direct_pardiso: FinishedS

In [39]:
wmg.Sessions

#0: LinslvPerfSer	XdgStokes-J512_p3_pMultigrid	04/07/2022 14:46:37	c31712d8...
#1: LinslvPerfSer	XdgStokes-J13824_p5_pMultigrid*	04/07/2022 15:00:32	9955f3f6...
#2: LinslvPerfSer	XdgStokes-J4096_p5_pMultigrid*	04/07/2022 14:58:10	3064dc8c...
#3: LinslvPerfSer	XdgStokes-J4096_p2_pMultigrid	04/07/2022 14:39:25	9a0cfcf5...
#4: LinslvPerfSer	XdgStokes-J512_p5_pMultigrid*	04/07/2022 14:55:50	c69cdf60...
#5: LinslvPerfSer	XdgStokes-J32768_p3_pMultigrid*	04/07/2022 14:53:32	dcd8b0c3...
#6: LinslvPerfSer	XdgStokes-J13824_p3_pMultigrid*	04/07/2022 14:51:15	cea6bbb7...
#7: LinslvPerfSer	XdgStokes-J4096_p3_pMultigrid*	04/07/2022 14:48:55	f9cfbc55...
#8: LinslvPerfSer	XdgStokes-J13824_p2_exp_Kcycle_schwarz	04/07/2022 14:13:55	485762d6...
#9: LinslvPerfSer	XdgStokes-J13824_p2_direct_pardiso	04/07/2022 13:29:52	2802c099...
#10: LinslvPerfSer	XdgStokes-J32768_p2_pMultigrid*	04/07/2022 14:44:19	aa190bac...
#11: LinslvPerfSer	XdgStokes-J13824_p2_pMultigrid*	04/07/2022 14:41:53	811ae86f...
#12: LinslvPe

In [43]:
//wmg.Sessions[118]

In [31]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

#0: BottiPietroStokes2D-J64_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: BottiPietroStokes2D-J256_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: BottiPietroStokes2D-J576_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: BottiPietroStokes2D-J1024_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#4: BottiPietroStokes2D-J2304_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#5: BottiPietroStokes2D-J4096_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#6: BottiPietroStokes2D-J16384_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#7: BottiPietroStokes2D

In [32]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);
    //Console.WriteLine(fail.LatestDeployment);

BottiPietroStokes2D-J64_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093324
BottiPietroStokes2D-J256_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093417
BottiPietroStokes2D-J576_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093509
BottiPietroStokes2D-J1024_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093603
BottiPietroStokes2D-J2304_p5_exp_gmres_levelpmg: FailedOrCanceled (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests):  @\\fdygitrunner\Va

In [33]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes2D") && Si.SuccessfulTermination == false);
FailedSessions

#0: LinslvPerfSer	BottiPietroStokes2D-J147456_p2_pMultigrid*	03/22/2022 10:21:49	4386d138...
#1: LinslvPerfSer	BottiPietroStokes2D-J65536_p2_pMultigrid*	03/22/2022 10:20:25	01b67a24...
#2: LinslvPerfSer	BottiPietroStokes2D-J36864_p2_pMultigrid*	03/22/2022 10:19:12	58135ee1...
#3: LinslvPerfSer	BottiPietroStokes2D-J16384_p2_pMultigrid*	03/22/2022 10:17:58	30f8b6d7...
#4: LinslvPerfSer	BottiPietroStokes2D-J2304_p2_pMultigrid*	03/22/2022 10:15:28	e103c5af...
#5: LinslvPerfSer	BottiPietroStokes2D-J36864_p5_exp_gmres_levelpmg*	03/22/2022 09:40:11	774bb1d0...
#6: LinslvPerfSer	BottiPietroStokes2D-J16384_p5_exp_gmres_levelpmg*	03/22/2022 09:39:13	61a7c9bd...
#7: LinslvPerfSer	BottiPietroStokes2D-J4096_p5_exp_gmres_levelpmg*	03/22/2022 09:38:18	1d3e42d7...
#8: LinslvPerfSer	BottiPietroStokes2D-J2304_p5_exp_gmres_levelpmg*	03/22/2022 09:37:21	e4c2227f...
#9: LinslvPerfSer	BottiPietroStokes2D-J1024_p5_exp_gmres_levelpmg*	03/22/2022 09:36:28	b8eeb77d...
#10: LinslvPerfSer	BottiPietroStokes2D-J576

In [34]:
FailedSessions.Select(si => (si, si.GetSessionDirectory(), si.DeployPath))

index,Item1,Item2,Item3
0,LinslvPerfSer BottiPietroStokes2D-J147456_p2_pMultigrid* 03/22/2022 10:21:49 4386d138...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\4386d138-7ff9-4fc3-848c-8cdb40d883f4,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_102118
1,LinslvPerfSer BottiPietroStokes2D-J65536_p2_pMultigrid* 03/22/2022 10:20:25 01b67a24...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\01b67a24-e72c-4990-b1a5-2a044befb20b,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_102000
2,LinslvPerfSer BottiPietroStokes2D-J36864_p2_pMultigrid* 03/22/2022 10:19:12 58135ee1...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\58135ee1-e39e-4255-8187-64b7858a4332,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_101844
3,LinslvPerfSer BottiPietroStokes2D-J16384_p2_pMultigrid* 03/22/2022 10:17:58 30f8b6d7...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\30f8b6d7-57e2-426e-b8c4-e172575201e6,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_101727
4,LinslvPerfSer BottiPietroStokes2D-J2304_p2_pMultigrid* 03/22/2022 10:15:28 e103c5af...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\e103c5af-467e-4109-b75e-3220d76b419e,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_101459
5,LinslvPerfSer BottiPietroStokes2D-J36864_p5_exp_gmres_levelpmg* 03/22/2022 09:40:11 774bb1d0...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\774bb1d0-8506-447f-96d9-beb91412e57e,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093945
6,LinslvPerfSer BottiPietroStokes2D-J16384_p5_exp_gmres_levelpmg* 03/22/2022 09:39:13 61a7c9bd...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\61a7c9bd-0951-4c92-a53d-74a3bcbffa67,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093847
7,LinslvPerfSer BottiPietroStokes2D-J4096_p5_exp_gmres_levelpmg* 03/22/2022 09:38:18 1d3e42d7...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\1d3e42d7-c807-4bc3-884d-651404605531,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093751
8,LinslvPerfSer BottiPietroStokes2D-J2304_p5_exp_gmres_levelpmg* 03/22/2022 09:37:21 e4c2227f...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\e4c2227f-6d18-49c4-a3ed-83c668a68830,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093657
9,LinslvPerfSer BottiPietroStokes2D-J1024_p5_exp_gmres_levelpmg* 03/22/2022 09:36:28 b8eeb77d...,\\fdygitrunner\ValidationTests\LinslvPerfSer\sessions\b8eeb77d-0889-4b7f-ad78-971456c14ca6,\\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022Mar22_093603


In [35]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [36]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

Unhandled exception: NUnit.Framework.AssertionException:   Some Jobs Failed
  Expected: 0
  But was:  13

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#38.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catch

In [37]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");

Unhandled exception: NUnit.Framework.AssertionException:   Some Sessions did not terminate successfully.
  Expected: 0
  But was:  13

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.Zero(Int32 actual, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Conditions.cs:line 484
   at Submission#39.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exc